In [138]:
import pandas as pd
from pymongo import MongoClient
from bson.son import SON
from bson.code import Code
import re
import pprint


client = MongoClient('localhost',27017)
db = client.movie_lens

In [139]:
print(db.list_collection_names())

['tags', 'links', 'movie_collector', 'genre_count', 'tag_count', 'genome_tags', 'movies', 'avg_ratings', 'ratings', 'genome_scores']


In [142]:
def similarity(I,J):
    intersection=db.ratings.aggregate([
        {
            '$match':{
                '$or':[
                    {'userId':I},
                    {'userId':J}
                ]
            }
        },
        {
            '$group':{
                '_id':'$movieId',
                'count':{'$sum':1}
            }
        },
        {
            '$match':{
                'count':{'$eq':2}
            }
        }
    ])
    count=0
    for i in intersection:
        count+=1
    return count


In [143]:
similarity(1,2)

4

In [144]:
def dissimilarity(I,J):
    union=db.ratings.aggregate([
        {
            '$match':{
                '$or':[
                    {'userId':I},
                    {'userId':J}
                ]
            }
        },
        {
            '$group':{
                '_id':'$movieId',
                'count':{'$sum':1}
            }
        }
    ])
    union_IJ=0
    for i in union:
        union_IJ+=1
    
    intersection=db.ratings.aggregate([
        {
            '$match':{
                '$or':[
                    {'userId':I},
                    {'userId':J}
                ]
            }
        },
        {
            '$group':{
                '_id':'$movieId',
                'count':{'$sum':1}
            }
        },
        {
            '$match':{
                'count':{'$eq':2}
            }
        }
    ])
    intersection_IJ=0
    for i in intersection:
        intersection_IJ+=1
    
    return union_IJ-intersection_IJ

In [145]:
dissimilarity(1,2)

246

In [26]:
m=162541

##### def summation_similarity_I(I):
#####     summation=0
#####     for i in range(1,m+1):
#####         summation+=similarity(I,i)
#####     return summation 
##### 무지성으로 코딩을하니 존나 오래걸렸다 mapreduce가 강제되는데 이걸 어캐하냐 

In [50]:
mapper=Code('''
    function() {
        emit(this.movieId,1);
    };
''')

In [51]:
reducer=Code('''
    function(key,values) {
        var total=0;
        for(var i=0;i<values.length;i++){
            total+=values[i];
        }
        return total;
    };
''')

In [52]:
db.ratings.map_reduce(mapper,reducer,'movie_collector')

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'movie_lens'), 'movie_collector')

In [146]:
def summation_similarity_I(I):
    cur=db.ratings.aggregate([
        {
            '$match':{'userId':I}
        },
        {
            '$lookup':{
                'from':'movie_collector',
                'localField':'movieId',
                'foreignField':'_id',
                'as':'counting'
            }
        },
        {
            '$project':{
                'userId':1,
                'movieId':1,
                'counting':'$counting.value'          
            }     
        },
        {
            '$unwind':'$counting'
        },
        {
            "$group": {
                "_id": "$userId",
                "count": {
                    "$sum": '$counting'
                }
            }
        }]
    )
    return list(cur)[0]['count']

In [149]:
def hat_similarity_IJ(I,J):
    return similarity(I,J)/summation_similarity_I(I)

In [151]:
hat_similarity_IJ(1,3)

1.829704419567857e-05

In [200]:
def gamma_I(I):    
    cur=db.ratings.aggregate([
        {
            '$match':{'userId':I}
        },
        {
            '$group':{
                '_id':'$userId',
                'count':{
                    '$sum':1
                }
            }
        }
    ])
    gamma_I=list(cur)[0]['count']
    return gamma_I

In [201]:
# def gamma_I(I):
#     return db.ratings.find({'userId':1}).count()

In [202]:
x=gamma_I(1)

In [203]:
print(x)

70


In [217]:
cur=db.ratings.aggregate([
    {
        '$group':{
            '_id':'$userId'
        }
    },
    {
        '$group':{
            '_id':'null',
            'count':{
                '$sum':1
            }
        }
    },
    {
        '$sort':{'_id':1}
    }
])
num_users=list(cur)[0]['count']

In [220]:
print(num_users)

162541


In [221]:
cur=db.ratings.aggregate([
    {
        '$group':{
            '_id':'null',
            'count':{
                '$sum':1
            }
        }
    }
])
num_ratings=list(cur)[0]['count']

In [222]:
print(num_ratings)

25000095


In [227]:
def summation_dissimilarity_I(I):
    summation=0
    summation+=(num_users*gamma_I(I))
    summation+=num_ratings
    summation-=(2*summation_similarity_I(I))
    
    return summation

In [231]:
summation_dissimilarity_I(1)

35175585.0

In [229]:
def hat_dissimilarity_IJ(I,J):
    return dissimilarity(I,J)/summation_dissimilarity_I(I)

In [230]:
hat_dissimilarity_IJ(1,2)

6.993487101920266e-06